## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [30]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 声明模型对应的参数
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(1)
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        logits = self.dense3(x)
        return logits


model = myModel()

optimizer = optimizers.Adam()

In [34]:
print(model.dense1.weights)

[<tf.Variable 'dense_16/kernel:0' shape=(784, 128) dtype=float32, numpy=
array([[-0.04329433,  0.06469939,  0.04489004, ...,  0.06226438,
        -0.0543303 , -0.06278146],
       [-0.06913303,  0.0759811 ,  0.01304957, ..., -0.0049606 ,
         0.00575867,  0.02136768],
       [ 0.01566284, -0.01908249,  0.03143829, ..., -0.05631116,
        -0.06291384,  0.07901058],
       ...,
       [-0.0776336 ,  0.01192926,  0.04566541, ...,  0.04589302,
         0.00315764,  0.00827827],
       [-0.07151937, -0.06119707,  0.03209096, ..., -0.07298043,
        -0.01579071, -0.04150775],
       [-0.05206762, -0.05984239, -0.0535663 , ..., -0.00416112,
        -0.06961697, -0.05746459]], dtype=float32)>, <tf.Variable 'dense_16/bias:0' shape=(128,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

## 计算 loss

In [31]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [15]:
train_data, test_data = mnist_dataset()
print(train_data[0].shape)
print(train_data[1].shape)

(60000, 28, 28)
(60000,)


In [32]:
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

AttributeError: in user code:

    File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_14004\2382397481.py", line 19, in train_one_step  *
        trainable_vars = [model.W1, model.W2, model.b1, model.b2]

    AttributeError: 'myModel' object has no attribute 'W1'
